# core

> Fill in a module description here

In [1]:
#| default_exp core

In [2]:
#| hide
from nbdev.showdoc import *

In [3]:
#|export
import numpy as np
from arch import arch_model
from functools import cached_property

In [4]:
#|export
class GARCHETTE:
    
    def __init__(self) -> None:
        self.params = None
        self._is_fit = False
        self._sig2_init = None

    def fit(self,y):
        self._y = y
        res = arch_model(y,mean='zero',dist='t').fit(disp='off')
        self.params = res.params
        self._sig2_init = res.conditional_volatility[0]**2
        self._is_fit = True
        return self
    
    @cached_property
    def vs(self):
        ...

    @cached_property
    def std_resid(self):
        return self._y / np.sqrt(self.vs)

    def simulate(self,horizon:int, n_rep=1000):
        ...

In [5]:
y = np.random.randn(300)
GARCHETTE().fit(y)

In [6]:
#| hide
import nbdev; nbdev.nbdev_export()